In [43]:
import os

import torch
from transformers import DetrImageProcessor, DetrForObjectDetection

import utils

In [2]:
image_sets_path = '/mnt/data/DatasetsML/CV/VOC_2012/VOC2012/ImageSets/Main/'
train_path = os.path.join(image_sets_path, 'train.txt')
val_path = os.path.join(image_sets_path, 'val.txt')

images_path = '/mnt/data/DatasetsML/CV/VOC_2012/VOC2012/JPEGImages/'
annnots_path = '/mnt/data/DatasetsML/CV/VOC_2012/VOC2012/Annotations/'

train_images = utils.read_set(train_path)
val_images = utils.read_set(val_path)

In [3]:
processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50")
model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50")

The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.
Some weights of the model checkpoint at facebook/detr-resnet-50 were not used when initializing DetrForObjectDetection: ['model.backbone.conv_encoder.model.layer1.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing DetrForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DetrForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a Ber

In [4]:
print(f"Model size: {utils.model_size(model):.3f} MB")

Model size: 158.404 MB


# Post Training Dynamic Quantization

In [37]:
model_int8 = torch.quantization.quantize_dynamic(
    model,
    {torch.nn.Linear},
    dtype=torch.qint8,
)

In [38]:
print(f"Model size: {utils.model_size(model_int8):.3f} MB")

Model size: 91.633 MB


In [39]:
_ = model_int8.cuda()

In [40]:
ptdq_res_map, ptdq_mean_sec_per_frame = utils.evaluate_over_voc(
    images_path=images_path,
    annots_path=annnots_path,
    val_images=val_images,
    model=model_int8,
    proc=processor,
)

  0%|          | 0/5823 [00:00<?, ?it/s]

NotImplementedError: Could not run 'quantized::linear_dynamic' with arguments from the 'CUDA' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'quantized::linear_dynamic' is only available for these backends: [CPU, BackendSelect, Python, FuncTorchDynamicLayerBackMode, Functionalize, Named, Conjugate, Negative, ZeroTensor, ADInplaceOrView, AutogradOther, AutogradCPU, AutogradCUDA, AutogradXLA, AutogradMPS, AutogradXPU, AutogradHPU, AutogradLazy, AutogradMeta, Tracer, AutocastCPU, AutocastCUDA, FuncTorchBatched, FuncTorchVmapMode, Batched, VmapMode, FuncTorchGradWrapper, PythonTLSSnapshot, FuncTorchDynamicLayerFrontMode, PythonDispatcher].

CPU: registered at ../aten/src/ATen/native/quantized/cpu/qlinear_dynamic.cpp:662 [kernel]
BackendSelect: fallthrough regis

In [ ]:
ptdq_mean_sec_per_frame

In [ ]:
ptdq_res_map